In [ ]:
import os
import json
import subprocess
import itertools

: 

In [ ]:
i_prefechers = ["no_instr", "next_line_instr"]

prefetchers = [ "no", "spp_dev", "va_ampm_lite", "ip_stride", "next_line"]

replacements = ["drrip", "lru", "ship", "srrip"]

In [ ]:
change_list = ["L1D", "L2C", "LLC"]

In [ ]:
# TODO: 修改trace路径，使用绝对路径
trace_path = "/home/zzx/文档/ChampSim/traces/400.perlbench-41B.champsimtrace.xz"

In [ ]:
# 备份一下
os.system("cp champsim_config.json my_champsim_config.json")

In [ ]:
with open("my_champsim_config.json", "r", encoding="utf-8") as f:
    config = json.load(f)

In [ ]:
# config["L1I"]["replacement"] = "lru"

In [ ]:
# with open("my_champsim_config.json", "w", encoding="utf-8") as f:
#     json.dump(config, f, ensure_ascii=False, indent=4)

In [ ]:
# os.system("./config.sh my_champsim_config.json")

In [ ]:
# os.system("make")

In [ ]:
# res = subprocess.getoutput("bin/champsim --warmup-instructions 10000000 --simulation-instructions 20000000 /home/zzx/文档/ChampSim/traces/400.perlbench-41B.champsimtrace.xz")

In [ ]:
# with open("results/test.log", "w+", encoding="utf-8") as f:
    # f.write(res)

In [ ]:
import signal

class TimeOutException(BaseException):
    pass

def set_timeout(seconds):
    signal.signal(signal.SIGALRM, timeout_hadler)
    signal.alarm(seconds)

def timeout_hadler(signum, frame):
    raise TimeOutException("运行超时")

cannot_run = []

In [ ]:
logs = sorted(os.listdir("results"))
logs = [dir for dir in logs if os.path.splitext(dir)[1] == '.log']
falid_file = []
for name in logs:
    with open("results/"+name, "r", encoding="utf-8") as f:
        data = f.readlines()
    if len(data) == 0:
        falid_file.append(name)

In [ ]:
for name in falid_file:
    ipref = name.split("-")[0]
    pref = name.split("-")[1]
    rep = name.split("-")[2].replace(".log", "")
    config["L1I"]["prefetcher"] = ipref
    config["L1I"].pop("replacement", None)
    for i in change_list:
        config[i]["prefetcher"] = pref
        config[i].pop("replacement", None)
    config[i]["replacement"] = rep
    
    with open("my_champsim_config.json", "w", encoding="utf-8") as f:
        json.dump(config, f, ensure_ascii=False, indent=4)
    
    res = os.system("./config.sh my_champsim_config.json")
    res = subprocess.getoutput("make")
    try:
        # TODO: 设置运行超时时间，单位为秒，在我的电脑上运行一次需要3~4分钟，所以我这里将超时时间设置为6分钟
        set_timeout(6 * 60)
        # TODO: 下面的指令数也可以换成别的
        res = subprocess.getoutput(f"bin/champsim --warmup-instructions 10000000 --simulation-instructions 20000000 {trace_path}")
        signal.alarm(0)
        print(f"运行成功: {ipref} {pref} {rep}")
        with open(f"results/{ipref}-{pref}-{rep}.log", "w+", encoding="utf-8") as f:
            f.write(res)
    except TimeOutException as e:
        print(e, end='')
        print(f": {ipref} {pref} {rep}")
        cannot_run.append((ipref, pref, rep))
        with open(f"results/{ipref}-{pref}-{rep}.log", "w+", encoding="utf-8") as f:
            pass

在开始执行前最好先跑一遍默认配置，记录正常运行时间，修改下面的超时时间。

In [ ]:
res_log_list = []
if os.path.exists("results"):
    res_log_list = os.listdir("results")
    

In [ ]:
for ipref, pref, rep in itertools.product(i_prefechers, prefetchers, replacements):
    file_name = f"{ipref}-{pref}-{rep}.log"
    if file_name in res_log_list:
        print("已经存在", f": {ipref} {pref} {rep}",sep='')
        continue
    config["L1I"]["prefetcher"] = ipref
    config["L1I"]["replacement"] = rep
    for i in change_list:
        config[i]["prefetcher"] = pref
        config[i]["replacement"] = rep
    
    with open("my_champsim_config.json", "w", encoding="utf-8") as f:
        json.dump(config, f, ensure_ascii=False, indent=4)
    
    res = os.system("./config.sh my_champsim_config.json")
    res = subprocess.getoutput("make")
    try:
        # TODO: 设置运行超时时间，单位为秒，在我的电脑上运行一次需要3~4分钟，所以我这里将超时时间设置为6分钟
        set_timeout(6 * 60)
        # TODO: 下面的指令数也可以换成别的
        res = subprocess.getoutput(f"bin/champsim --warmup-instructions 10000000 --simulation-instructions 20000000 {trace_path}")
        signal.alarm(0)
        print(f"运行成功: {ipref} {pref} {rep}")
        with open(f"results/{ipref}-{pref}-{rep}.log", "w+", encoding="utf-8") as f:
            f.write(res)
    except TimeOutException as e:
        print(e, end='')
        print(f": {ipref} {pref} {rep}")
        cannot_run.append((ipref, pref, rep))
        with open(f"results/{ipref}-{pref}-{rep}.log", "w+", encoding="utf-8") as f:
            pass

print("falied num: ",len(cannot_run))

读取日志文件，收集相关信息

In [ ]:
logs = sorted(os.listdir("results"))
logs = [dir for dir in logs if os.path.splitext(dir)[1] == '.log']

In [ ]:
with open("results/"+logs[21], "r", encoding="utf-8") as f:
    data = f.readlines()

In [ ]:
for row in data:
    if "CPU 0 cumulative IPC:" in row:
        res = row.split()
        print("IPC:", res[4])
    elif "cpu0_L1D TOTAL" in row:
        res = row.split()
        LLC_hit = int(res[5]) / int(res[3])
        print("cpu0_L1D hitrate:", LLC_hit)
    elif "cpu0_L1I TOTAL" in row:
        res = row.split()
        LLC_hit = int(res[5]) / int(res[3])
        print("cpu0_L1I hitrate:", LLC_hit)
    elif "cpu0_L2C TOTAL" in row:
        res = row.split()
        LLC_hit = int(res[5]) / int(res[3])
        print("cpu0_L2C hitrate:", LLC_hit)
    elif "LLC TOTAL" in row:
        res = row.split()
        LLC_hit = int(res[5]) / int(res[3])
        print("LLC hitrate:", LLC_hit)

In [ ]:
import csv
all = []

In [ ]:
for name in logs:
    with open("results/"+name, "r", encoding="utf-8") as f:
        data = f.readlines()
    if len(data) == 0:
        continue
    info = {"i_prefetcher": name.split("-")[0], "prefetcher": name.split("-")[1], "replacement": name.split("-")[2].replace(".log", "")}
    for row in data:
        if "CPU 0 cumulative IPC:" in row:
            res = row.split()
            # print("IPC:", res[4])
            info["IPC"] = res[4]
        elif "cpu0_L1D TOTAL" in row:
            res = row.split()
            LLC_hit = int(res[5]) / int(res[3])
            # print("cpu0_L1D hitrate:", LLC_hit)
            info["L1D"] = "{:.2f}".format(LLC_hit * 100)
        elif "cpu0_L1I TOTAL" in row:
            res = row.split()
            LLC_hit = int(res[5]) / int(res[3])
            # print("cpu0_L1I hitrate:", LLC_hit)
            info["L1I"] = "{:.2f}".format(LLC_hit * 100)
        elif "cpu0_L2C TOTAL" in row:
            res = row.split()
            LLC_hit = int(res[5]) / int(res[3])
            # print("cpu0_L2C hitrate:", LLC_hit)
            info["L2C"] = "{:.2f}".format(LLC_hit * 100)
        elif "LLC TOTAL" in row:
            res = row.split()
            LLC_hit = int(res[5]) / int(res[3])
            # print("LLC hitrate:", LLC_hit)
            info["LLC"] = "{:.2f}".format(LLC_hit * 100)
    all.append(info)

In [ ]:
with open("results/data.csv", "w+", encoding="utf-8", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["L1I prefetcher", "prefetcher", "replacement", "IPC", "L1D", "L1I", "L2C", "LLC"])
    for info in all:
        csv_writer.writerow([info["i_prefetcher"], info["prefetcher"], info["replacement"],info["IPC"],info["L1D"],info["L1I"],info["L2C"],info["LLC"]])

In [ ]:
header = ["L1I预取策略", "预取策略", "替换策略", "IPC", "L1D", "L1I", "L2C", "LLC"]
print("| ", end='')
for i in header:
    print(i, "| ", end='')
print("")
print("|", end='')
for i in range(len(header)):
    print("----|",end='')
print("")
for info in all:
    row = [info["i_prefetcher"], info["prefetcher"], info["replacement"],info["IPC"],info["L1D"],info["L1I"],info["L2C"],info["LLC"]]
    print("| ", end='')
    for i in row:
        print(i, "| ", end='')
    print("")

将markdown表格形式写入文件

In [ ]:
with open("results/res.md", "w+", encoding="utf-8") as f:
    print("| ", end='', file=f)
    for i in header:
        print(i, "| ", end='', file=f)
    print("", file=f)
    print("|", end='', file=f)
    for i in range(len(header)):
        print("----|",end='', file=f)
    print("", file=f)
    for info in all:
        row = [info["i_prefetcher"], info["prefetcher"], info["replacement"],info["IPC"],info["L1D"],info["L1I"],info["L2C"],info["LLC"]]
        print("| ", end='', file=f)
        for i in row:
            print(i, "| ", end='', file=f)
        print("", file=f)